In [1]:
!pip install streamlit

In [2]:
!pip install pyngrok

In [3]:
!pip install yfinance --upgrade --no-cache-dir

Requirement already up-to-date: yfinance in /usr/local/lib/python3.6/dist-packages (0.1.55)


In [25]:
%%writefile appPut.py
#Bibliotecas
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader.data as web
import streamlit as st

yf.pdr_override()

#Input da ação a ser estudada
symbol = st.text_input("Ativo", "PETR4")


#Download dados
symbol = symbol + ".SA"

symbols = []

symbols.append(symbol)

df = web.get_data_yahoo(symbols)

portfolio = web.get_data_yahoo(symbols,start='2010-01-01')['Adj Close']
symbols.append('BOVA11.SA')
bova = web.get_data_yahoo(symbols,start='2010-01-01')['Adj Close']


#Calcula retornos
retornos = portfolio.pct_change().dropna()
bovaret = bova.pct_change().dropna()



#Calculo do Beta
beta = bovaret.cov()/bovaret['BOVA11.SA'].var()

#Input períodos do Desvio Padrão
N = st.number_input("Períodos  Desvio Padrão", 180)


# Desvio Padrao de N dias
desvios = retornos.rolling(N).std()

#Média dos desvios
mediaDP = desvios.mean()

#Maior valor dos desvios
maiorDP = desvios.max()

#Menor valor dos desvios
menorDP = desvios.min()

retorno = pd.DataFrame()

retorno['Retorno %'] = retornos * 100

#st.write("Ativo estudado", symbols[:-3])
st.write("Desvio Padrão Médio:", round(mediaDP * 100,2),"%")
st.write("Desvio Padrão Máx:", round(maiorDP * 100,2),"%")
st.write("Desvio Padrão Min:", round(menorDP *100,2),"%")
st.write("Beta em relação ao BOVA11:", round(beta[symbol][0],2))
st.write("")
st.write("Retornos")
st.line_chart(retorno)

st.write("")
st.write("Diferenças Sexta-Sexta")
#Identificar a 3 segunda feira do mes seguinte
#cria coluna com as segundas-feiras
df['segunda'] = df['Adj Close'].index.weekday==0

#Sextas
df['sexta'] = df['Adj Close'].index.weekday==4
#cria coluna com as datas
df['data'] = df.index
#Marca de 1 a 5 qual é a semana do mes
df['wom'] = df['data'].apply(lambda d: (d.day-1) // 7 + 1)
#Cria coluna com o valor de fechamento da sexta-feira antes do vencimento
df['Sexta_close'] = df.query('sexta==True and wom==2')['Adj Close']
df.dropna(inplace=True)
#Calcular a diferença % entre uma sexta antes do vencimento e outra
periodos = 1
dfperc = df['Sexta_close'].pct_change(periodos)
st.write(dfperc.describe(percentiles=[0.05,0.1, 0.07, 0.15, 0.2, 0.22, 0.25,0.3,0.35, 0.4,0.5,0.6,0.7,0.8, 0.85,0.9,0.95, 0.96]))




Overwriting appPut.py


In [5]:
#Conexao com o ngrok
!ngrok authtoken 1nZ2OPZ73LxxYTtv9rQvaAKqpDn_4rZvztMGek7NAd6AmZWTX

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
from pyngrok import ngrok

In [7]:
!streamlit run --server.port 80 appPut.py&>/dev/null&

In [8]:
pub_url = ngrok.connect(port='8051')

In [9]:
pub_url

<NgrokTunnel: "http://c47824089842.ngrok.io" -> "http://localhost:80">